In [4]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

# Epsilon Metric

In [5]:
def load_pkl_dict(filename):
    folder = "../experiment_data/"
    with open(folder + filename, "rb") as f:
        loaded_file = pickle.load(f)
        return loaded_file

In [6]:
inspect = load_pkl_dict("oscr_data_base_1_1_-1_ABS_REV_ACTV_N-CLASSES.pkl")[0]
inspect = inspect['1500-1.25']
ccr = inspect[0]
fpr = inspect[1]
print(f"CCR (Y-axis): {ccr}")
print(f"FPR (X-axis) non-log: {fpr}")
print(f"FPR (X-axis) log: {np.log(fpr)}")
print(f"Shape: CCR {ccr.shape} and FPR {fpr.shape}")

threshy = 0.6
ar_diff = np.abs(fpr - threshy)
idx = np.argmin(ar_diff)
print(f"At threshold {threshy} actual value in fpr {fpr[idx]} with corresponding CCR {ccr[idx]}")

def oscr_epsilon_ccr_at_fpr(ccr, fpr, fpr_thresholds):
    threshold_values = []
    for threshold in fpr_thresholds:
        idx = np.where(fpr==threshold)
        value = ccr[idx]
        if value.shape[0] == 0: 
            value = np.append(value, 0.0)
        threshold_values.append((threshold, value))
    return np.sum([t[1] for t in threshold_values]), threshold_values

t_list = [0.1, 0.2, 0.3, 0.4, 0.8]

eps, t_val = oscr_epsilon_ccr_at_fpr(ccr, fpr, t_list)
eps, t_val

KeyError: 0

In [7]:
# Assume x is your [1000, 50] tensor
x = torch.randn(1000, 50)
print(x[0])
# Reshape the tensor to [1000, 10, 5]
x_reshaped = x.view(1000, 10, 5)
print(x_reshaped[0])

tensor([-0.6145, -0.1009, -0.6413,  0.6315, -1.8611, -2.6008,  0.1207,  1.4215,
        -0.9523,  0.8357,  0.2424, -0.5427, -1.7616, -1.9449,  1.1319, -1.3818,
         1.3361,  1.1566, -0.7354, -0.4909, -1.6611,  0.0519, -1.3816,  0.2349,
         0.8422, -1.1532, -0.4596, -0.2842,  1.0712, -0.6280, -0.3555, -0.4469,
         0.8330,  0.8367, -0.8764,  0.0221, -1.2065, -2.2033, -0.5871, -1.2283,
         0.3053,  0.0940, -1.7790,  0.0072, -1.0865, -0.3080, -0.4726, -0.5434,
        -0.5939, -0.6636])
tensor([[-0.6145, -0.1009, -0.6413,  0.6315, -1.8611],
        [-2.6008,  0.1207,  1.4215, -0.9523,  0.8357],
        [ 0.2424, -0.5427, -1.7616, -1.9449,  1.1319],
        [-1.3818,  1.3361,  1.1566, -0.7354, -0.4909],
        [-1.6611,  0.0519, -1.3816,  0.2349,  0.8422],
        [-1.1532, -0.4596, -0.2842,  1.0712, -0.6280],
        [-0.3555, -0.4469,  0.8330,  0.8367, -0.8764],
        [ 0.0221, -1.2065, -2.2033, -0.5871, -1.2283],
        [ 0.3053,  0.0940, -1.7790,  0.0072, -1.0865]

In [8]:
# Use torch.max on the last dimension
output, _ = torch.max(x_reshaped, dim=2)
output[0]

tensor([ 0.6315,  1.4215,  1.1319,  1.3361,  0.8422,  1.0712,  0.8367,  0.0221,
         0.3053, -0.3080])

In [9]:
def multiply_tensors_with_sign(sorted_activations, weights):
    mask = sorted_activations < 0
    weights[mask] = 2 - weights[mask]
    return sorted_activations * weights

In [10]:
def value_shift(sorted_activations):
    min_values = torch.min(sorted_activations, dim=1).values

    # Reshape the min_values tensor to match the shape of 'a' for broadcasting
    min_values_reshaped = min_values.view(-1, 1)
    min_values_reshaped_abs = torch.abs(min_values_reshaped)

    # Add the minimum values to the corresponding rows of the original tensor
    return torch.add(sorted_activations, min_values_reshaped_abs)

In [11]:
randy = torch.randn(5,5)
randy

tensor([[-0.3003, -1.0985,  0.2176, -0.7032, -2.2400],
        [ 0.8417,  1.0834, -0.6544, -0.6773,  0.9454],
        [-1.1860, -1.1562,  0.2957, -1.3506,  0.6772],
        [ 0.5114,  0.5328, -0.8902,  0.1429, -0.0282],
        [-0.7869,  1.2258,  0.6152,  0.8106, -0.6285]])

In [12]:
rando = torch.rand(5, 5)
rando

tensor([[0.7157, 0.7896, 0.1451, 0.8449, 0.9600],
        [0.1111, 0.0299, 0.5465, 0.1389, 0.5771],
        [0.7874, 0.4050, 0.6742, 0.8344, 0.1365],
        [0.3927, 0.5948, 0.4210, 0.2279, 0.8987],
        [0.0141, 0.0490, 0.6527, 0.3933, 0.5038]])

In [13]:
val = value_shift(randy) * rando

In [14]:
abs_value = multiply_tensors_with_sign(randy, rando)

In [15]:
softmax_abs = torch.nn.functional.softmax(abs_value, dim=1)
abs_value, softmax_abs, torch.max(softmax_abs, dim=1)

(tensor([[-0.3856, -1.3296,  0.0316, -0.8123, -2.3296],
         [ 0.0935,  0.0324, -0.9511, -1.2604,  0.5456],
         [-1.4381, -1.8442,  0.1994, -1.5742,  0.0924],
         [ 0.2009,  0.3169, -1.4057,  0.0326, -0.0310],
         [-1.5626,  0.0601,  0.4015,  0.3188, -0.9404]]),
 tensor([[0.2701, 0.1051, 0.4099, 0.1763, 0.0387],
         [0.2426, 0.2282, 0.0853, 0.0626, 0.3812],
         [0.0813, 0.0542, 0.4180, 0.0709, 0.3756],
         [0.2524, 0.2835, 0.0506, 0.2133, 0.2002],
         [0.0463, 0.2343, 0.3297, 0.3035, 0.0862]]),
 torch.return_types.max(
 values=tensor([0.4099, 0.3812, 0.4180, 0.2835, 0.3297]),
 indices=tensor([2, 4, 2, 1, 2])))

In [16]:
softmax_val = torch.nn.functional.softmax(val, dim=1)
val, softmax_val, torch.max(softmax_val, dim=1)

(tensor([[1.3882, 0.9013, 0.3565, 1.2984, 0.0000],
         [0.1687, 0.0526, 0.0125, 0.0000, 0.9365],
         [0.1296, 0.0787, 1.1099, 0.0000, 0.2768],
         [0.5505, 0.8464, 0.0000, 0.2354, 0.7747],
         [0.0000, 0.0986, 0.9151, 0.6283, 0.0798]]),
 tensor([[0.3190, 0.1961, 0.1137, 0.2916, 0.0796],
         [0.1740, 0.1550, 0.1489, 0.1470, 0.3751],
         [0.1503, 0.1429, 0.4006, 0.1320, 0.1742],
         [0.2040, 0.2742, 0.1176, 0.1489, 0.2553],
         [0.1323, 0.1460, 0.3304, 0.2480, 0.1433]]),
 torch.return_types.max(
 values=tensor([0.3190, 0.3751, 0.4006, 0.2742, 0.3304]),
 indices=tensor([0, 4, 2, 1, 2])))

In [17]:
total = torch.sum(val, dim=1)
total

tensor([3.9444, 1.1703, 1.5950, 2.4070, 1.7218])

In [18]:
matrix = np.arange(25).reshape((5,5))
matrix

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [19]:
maxi = np.max(matrix, axis=1)
maxi

array([ 4,  9, 14, 19, 24])

In [20]:
summe = np.sum(maxi)
summe

70

In [21]:
gt = np.arange(matrix.shape[0])
mask = gt > 0
mask, gt[mask], matrix[mask]

(array([False,  True,  True,  True,  True]),
 array([1, 2, 3, 4]),
 array([[ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]))

In [22]:
# Retrieve the values
values = matrix[mask][np.arange(matrix[mask].shape[0]), gt[mask]]
print(gt[mask])

print(values)

[1 2 3 4]
[ 6 12 18 24]


In [26]:
from datetime import datetime
print(time.asctime)
now = datetime.now()
print(now)

<built-in function asctime>
2023-11-20 19:36:31.707054
